<a href="https://colab.research.google.com/github/deltorobarba/science/blob/main/alicebob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantum Computing on Alice & Bob**

In [ ]:
!pip install -U qiskit-alice-bob-provider -q
from google.colab import userdata
from qiskit_alice_bob_provider import AliceBobRemoteProvider
from qiskit import QuantumCircuit, transpile

In [ ]:
# 1. Retrieve API key and connect to Alice & Bob
api_token = userdata.get('alicebob')
provider = AliceBobRemoteProvider(api_key=api_token)

In [ ]:
# 2. List available backends
for backend in provider.backends():
    print(backend.name)

EMU:1Q:LESCANNE_2020
QPU:1Q:BOSON_4A
QPU:1Q:BOSON_4B
QPU:1Q:BOSON_4C
EMU:6Q:PHYSICAL_CATS
EMU:40Q:PHYSICAL_CATS
EMU:40Q:LOGICAL_TARGET
EMU:40Q:LOGICAL_NOISELESS
EMU:15Q:LOGICAL_EARLY


In [ ]:
# 3. Select backend (QPU quantum or EMU emulator)
backend = provider.get_backend('EMU:6Q:PHYSICAL_CATS', average_nb_photons=10.0)
print(f"Selected backend: {backend.name}")

Selected backend: EMU:6Q:PHYSICAL_CATS


In [ ]:
# 4. Define Quantum Circuit
qc = QuantumCircuit(1, 1)
qc.initialize('1', 0)
qc.delay(duration=1, qarg=0, unit='ms')
qc.measure(0, 0) # Measures bit-flip probability of single qubit after a 1ms delay
print(qc.draw())

     ┌───────────────┐┌──────────────┐┌─┐
  q: ┤ Initialize(1) ├┤ Delay(1[ms]) ├┤M├
     └───────────────┘└──────────────┘└╥┘
c: 1/══════════════════════════════════╩═
                                       0 


In [ ]:
# 6. Transpile circuit to specific backend to ensure correct error-correction properties
transpiled = transpile(qc, backend)
print(transpiled.draw())

     ┌───────────────┐┌─────────────────┐┌─┐
   q ┤ Initialize(1) ├┤ Delay(0.001[s]) ├┤M├
     └───────────────┘└─────────────────┘└╥┘
c: 1/═════════════════════════════════════╩═
                                          0 


In [ ]:
# 7. Run circuit on Felis Cloud
job = backend.run(transpiled, shots=1000)

In [ ]:
# 8. Get results
res = job.result()
print(f"Job Status: {job.status()}")
print(f'Total counts: {res.get_counts()}')

Job Status: JobStatus.DONE
Total counts: {'1': 1000}


In [ ]:
# Console: https://api-gcp.alice-bob.com/console
# Backend: https://felis.alice-bob.com/docs/backends/about_backends/
# Status:  https://felis.alice-bob.com/docs/felis_cloud/hardware_availability_schedule/
# Samples: https://github.com/Alice-Bob-SW/felis/tree/main/samples